# 제 1 유형

> 데이터셋
* https://www.data.go.kr/data/15007122/fileData.do
* 2018년도 성인의 건강검진 데이터(흡연상태: 1-흡연, 0-비흡연)

In [1]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/train.csv")

## Q1.
> 시력(좌) 와 시력(우)의 값이 같은 남성의 허리둘레의 평균은?

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44553 entries, 0 to 44552
Data columns (total 26 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   성별코드         44553 non-null  object 
 1   연령대코드(5세단위)  44553 non-null  int64  
 2   신장(5Cm단위)    44553 non-null  int64  
 3   체중(5Kg단위)    44553 non-null  int64  
 4   허리둘레         44553 non-null  float64
 5   시력(좌)        44553 non-null  float64
 6   시력(우)        44553 non-null  float64
 7   청력(좌)        44553 non-null  float64
 8   청력(우)        44553 non-null  float64
 9   수축기혈압        44553 non-null  float64
 10  이완기혈압        44553 non-null  float64
 11  식전혈당(공복혈당)   44553 non-null  float64
 12  총콜레스테롤       44553 non-null  float64
 13  트리글리세라이드     44553 non-null  float64
 14  HDL콜레스테롤     44553 non-null  float64
 15  LDL콜레스테롤     44553 non-null  float64
 16  혈색소          44553 non-null  float64
 17  요단백          44553 non-null  float64
 18  혈청크레아티닌      44553 non-null  float64
 19  (혈청지

In [16]:
print(df[(df["성별코드"] == "M") & (df["시력(좌)"] == df["시력(우)"])]["허리둘레"].mean())

84.9768266281797


## Q2.
> 40대(연령대코드 40,45) 여성 중 '총콜레스테롤'값이 40대 여성의 '총콜레스테롤' 중간값 이상을 가지는 그룹과 50대(연령대코드 50,55) 여성 중 '총콜레스레롤'값이 50대 여성의 '총콜레스테롤' 중간값 이상을 가지는 두 그룹의 '수축기혈압'이 독립성, 정규성, 등분산성이 만족하는 것을 확인했다.<br>
두 그룹의 '수축기혈압'의 독립표본 t 검증 결과를 통계값, p-value 구분지어 구하라.

In [30]:
age_40 = df[((df["연령대코드(5세단위)"] == 40) | (df["연령대코드(5세단위)"] == 45)) & (df["성별코드"] == "F")]
age_50 = df[((df["연령대코드(5세단위)"] == 50) | (df["연령대코드(5세단위)"] == 55)) & (df["성별코드"] == "F")]

In [33]:
age_40_chol = age_40[age_40["총콜레스테롤"] >= age_40["총콜레스테롤"].median()]
age_50_chol = age_50[age_50["총콜레스테롤"] >= age_50["총콜레스테롤"].median()]

In [34]:
import scipy.stats as stats

t_test_result = stats.ttest_ind(age_40_chol["수축기혈압"], age_50_chol["수축기혈압"], equal_var=True, alternative="two-sided")
answer1 = t_test_result[0]
answer2 = t_test_result[1]

print(f"통계값: {answer1}")
print(f"p-value: {answer2}")

통계값: -8.954384087520708
p-value: 4.399762427897212e-19


# 제 2 유형

> 데이터셋
* https://www.data.go.kr/data/15007122/fileData.do
* 2018년도 성인의 건강검진 데이터로부터 흡연상태 예측(흡연상태: 1-흡연, 0-비흡연)
* 평가지표: f1-score

In [35]:
import pandas as pd
import numpy as np

train_df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/train.csv")
test_df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/test.csv")
submission_df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/submission.csv")

## 데이터 살펴보기

In [36]:
train_df

,성별코드,연령대코드(5세단위),신장(5Cm단위),체중(5Kg단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),수축기혈압,...,혈색소,요단백,혈청크레아티닌,(혈청지오티)AST,(혈청지오티)ALT,감마지티피,흡연상태,구강검진수검여부,치아우식증유무,치석
0,F,55,145,55,73.0,0.7,0.5,1.0,1.0,129.0,...,12.9,1.0,0.9,172.0,209.0,15.0,0,Y,0.0,Y
1,M,40,180,55,74.0,1.5,1.5,1.0,1.0,102.0,...,15.6,1.0,0.8,30.0,19.0,23.0,1,Y,0.0,Y
2,F,55,150,50,72.0,1.0,0.2,1.0,1.0,116.0,...,14.1,1.0,1.0,31.0,19.0,10.0,0,Y,0.0,N
3,M,40,170,85,89.0,1.0,0.8,1.0,1.0,124.0,...,16.7,1.0,1.0,26.0,38.0,108.0,1,Y,1.0,Y
4,F,40,155,45,62.0,0.5,1.0,1.0,1.0,127.0,...,12.9,1.0,0.7,20.0,11.0,13.0,0,Y,0.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44548,F,60,145,50,87.0,1.0,1.0,1.0,1.0,140.0,...,12.3,1.0,0.5,24.0,32.0,15.0,0,Y,1.0,Y
44549,M,40,175,75,88.0,1.5,1.2,1.0,1.0,139.0,...,15.3,1.0,0.8,62.0,106.0,49.0,1,Y,0.0,Y
44550,F,60,150,55,82.0,0.9,0.9,1.0,1.0,124.0,...,14.1,1.0,0.5,38.0,34.0,26.0,0,Y,0.0,Y
44551,M,40,170,50,69.0,0.4,1.2,1.0,1.0,117.0,...,14.2,1.0,0.9,22.0,17.0,55.0,1,Y,0.0,N


In [37]:
train_df.isna().sum()

성별코드           0
연령대코드(5세단위)    0
신장(5Cm단위)      0
체중(5Kg단위)      0
허리둘레           0
시력(좌)          0
시력(우)          0
청력(좌)          0
청력(우)          0
수축기혈압          0
이완기혈압          0
식전혈당(공복혈당)     0
총콜레스테롤         0
트리글리세라이드       0
HDL콜레스테롤       0
LDL콜레스테롤       0
혈색소            0
요단백            0
혈청크레아티닌        0
(혈청지오티)AST     0
(혈청지오티)ALT     0
감마지티피          0
흡연상태           0
구강검진수검여부       0
치아우식증유무        0
치석             0
dtype: int64

In [38]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44553 entries, 0 to 44552
Data columns (total 26 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   성별코드         44553 non-null  object 
 1   연령대코드(5세단위)  44553 non-null  int64  
 2   신장(5Cm단위)    44553 non-null  int64  
 3   체중(5Kg단위)    44553 non-null  int64  
 4   허리둘레         44553 non-null  float64
 5   시력(좌)        44553 non-null  float64
 6   시력(우)        44553 non-null  float64
 7   청력(좌)        44553 non-null  float64
 8   청력(우)        44553 non-null  float64
 9   수축기혈압        44553 non-null  float64
 10  이완기혈압        44553 non-null  float64
 11  식전혈당(공복혈당)   44553 non-null  float64
 12  총콜레스테롤       44553 non-null  float64
 13  트리글리세라이드     44553 non-null  float64
 14  HDL콜레스테롤     44553 non-null  float64
 15  LDL콜레스테롤     44553 non-null  float64
 16  혈색소          44553 non-null  float64
 17  요단백          44553 non-null  float64
 18  혈청크레아티닌      44553 non-null  float64
 19  (혈청지

## 전처리 함수 생성

In [46]:
from sklearn.preprocessing import StandardScaler

def my_preprocess(org_df, target="train", scaler=None):
    df = org_df.copy()
    df["성별코드"] = df["성별코드"].replace({"M": 1, "F": 0})
    df["구강검진수검여부"] = df["구강검진수검여부"].replace({"Y": 1, "N": 0})
    df["치석"] = df["치석"].replace({"Y": 1, "N": 0})

    if target == "train":
        scaler1 = StandardScaler()
        df = pd.DataFrame(scaler1.fit_transform(df), columns=df.columns)
        return df, scaler1
    elif target == "test":
        scaler1 = scaler
        df = pd.DataFrame(scaler1.transform(df), columns=df.columns)
        return df
    return None

## 검증데이터와 훈련데이터 분할

In [41]:
from sklearn.model_selection import train_test_split

X = train_df.drop("흡연상태", axis=1)
y = train_df["흡연상태"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, stratify=y, random_state=1)

In [44]:
y_train.sum() / y_train.count()

0.36701189598230033

## 분할데이터에 전처리 함수 적용

In [49]:
X_train_proc, scaler = my_preprocess(X_train, target="train", scaler=None)
X_val_proc = my_preprocess(X_val, target="test", scaler=scaler)
test_proc = my_preprocess(test_df, target="test", scaler=scaler)

## 분석 수행

In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.metrics import f1_score, confusion_matrix, accuracy_score

### 로지스틱 회귀

In [54]:
model_lr = LogisticRegression()
model_lr.fit(X_train_proc, y_train)
pred = model_lr.predict(X_val_proc)

lr_mat = confusion_matrix(y_val, pred)
lr_acc = accuracy_score(y_val, pred)
lr_f1 = f1_score(y_val, pred)

print(lr_mat)
print(lr_acc)
print(lr_f1)

[[6495 1965]
 [1520 3386]]
0.7392638036809815
0.6602320366578922


### 랜덤 포레스트

In [59]:
scores = []
for depth in range(8, 16):
    model_rf = RandomForestClassifier(max_depth=depth, random_state=1)
    model_rf.fit(X_train_proc, y_train)
    pred = model_rf.predict(X_val_proc)
    scores.append(f1_score(y_val, pred))

pd.DataFrame(scores, index=range(8, 16), columns=["f1"]).sort_values("f1", ascending=False)

,f1
10,0.682120
11,0.681317
9,0.680554
15,0.679674
14,0.679414
8,0.678619
12,0.677021
13,0.675438


In [60]:
model_rf = RandomForestClassifier(max_depth=10, random_state=1)
model_rf.fit(X_train_proc, y_train)
pred = model_rf.predict(X_val_proc)

rf_mat = confusion_matrix(y_val, pred)
rf_acc = accuracy_score(y_val, pred)
rf_f1 = f1_score(y_val, pred)

print(rf_mat)
print(rf_acc)
print(rf_f1)

[[6515 1945]
 [1360 3546]]
0.7527308095166841
0.682119842262191


### Gradient Boosting

In [61]:
scores = []
for depth in range(3, 11):
    model_gb = GradientBoostingClassifier(max_depth=depth, random_state=1)
    model_gb.fit(X_train_proc, y_train)
    pred = model_gb.predict(X_val_proc)
    scores.append(f1_score(y_val, pred))

pd.DataFrame(scores, index=range(3, 11), columns=['f1']).sort_values("f1", ascending=False)

,f1
3,0.682063
6,0.678043
5,0.677118
4,0.676636
8,0.670036
7,0.668979
10,0.666202
9,0.665801


In [62]:
model_gb = GradientBoostingClassifier(max_depth=3, random_state=1)
model_gb.fit(X_train_proc, y_train)
pred = model_gb.predict(X_val_proc)

gb_mat = confusion_matrix(y_val, pred)
gb_acc = accuracy_score(y_val, pred)
gb_f1 = f1_score(y_val, pred)

print(gb_mat)
print(gb_acc)
print(gb_f1)

[[6632 1828]
 [1421 3485]]
0.7569205446655694
0.6820628241510912


### AdaBoost

In [63]:
for rate in [0.1, 0.3, 0.5, 0.7, 1]:
    model_ab = AdaBoostClassifier(learning_rate = rate, random_state=1)
    model_ab.fit(X_train_proc, y_train)
    pred = model_ab.predict(X_val_proc)
    print(rate, f1_score(y_val, pred))

0.1 0.6542722281090453
0.3 0.6742292030250145
0.5 0.6776649746192893
0.7 0.6719952634695087
1 0.67622343054869


In [64]:
model_ab = AdaBoostClassifier(learning_rate=0.5, random_state=1)
model_ab.fit(X_train_proc, y_train)
pred = model_ab.predict(X_val_proc)

ab_mat = confusion_matrix(y_val, pred)
ab_acc = accuracy_score(y_val, pred)
ab_f1 = f1_score(y_val, pred)

print(ab_mat)
print(ab_acc)
print(ab_f1)

[[6593 1867]
 [1435 3471]]
0.7529552596139458
0.6776649746192893


### KNN

In [78]:
scores = []
for k in range(50, 70):
    model_knn = KNeighborsClassifier(n_neighbors=k)
    model_knn.fit(X_train_proc, y_train)
    pred = model_knn.predict(X_val_proc)
    scores.append(f1_score(y_val, pred))

pd.DataFrame(scores, index=range(50, 70), columns=["f1"]).sort_values("f1", ascending=False)

,f1
63,0.655658
69,0.653097
67,0.652953
65,0.652774
59,0.651816
55,0.651218
57,0.651140
61,0.650666
53,0.650373
51,0.649317


In [79]:
model_knn = KNeighborsClassifier(n_neighbors=63)
model_knn.fit(X_train_proc, y_train)
pred = model_knn.predict(X_val_proc)

knn_mat = confusion_matrix(y_val, pred)
knn_acc = accuracy_score(y_val, pred)
knn_f1 = f1_score(y_val, pred)

print(knn_mat)
print(knn_acc)
print(knn_f1)

[[6487 1973]
 [1551 3355]]
0.7363459524165794
0.655657611881962


### SVM

In [80]:
model_svm = SVC(kernel="rbf", C=0.1, random_state=1)
model_svm.fit(X_train_proc, y_train)
pred = model_svm.predict(X_val_proc)

svm_mat = confusion_matrix(y_val, pred)
svm_acc = accuracy_score(y_val, pred)
svm_f1 = f1_score(y_val, pred)

print(svm_mat)
print(svm_acc)
print(svm_f1)

[[6518 1942]
 [1490 3416]]
0.7432290887326051
0.6656274356975839


## 최종 모델로 test 데이터 예측

In [81]:
print(lr_f1)
print(rf_f1)
print(gb_f1)
print(ab_f1)
print(knn_f1)
print(svm_f1)

0.6602320366578922
0.682119842262191
0.6820628241510912
0.6776649746192893
0.655657611881962
0.6656274356975839


In [82]:
test_pred = model_rf.predict(test_proc)
submission_df["0"] = test_pred
submission_df.to_csv("submission_pred.csv")